In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt

In [8]:
from __future__ import print_function
import keras
from keras.models import load_model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras import applications
from keras.models import Model, load_model


from time import time

In [9]:
# Data generators, data augmentation
batch_size = 4

train = ImageDataGenerator(rotation_range = 45.0,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    shear_range = 30.,
    zoom_range = 0.3)

test = ImageDataGenerator(rotation_range = 45.0,
    zoom_range = 0.3)

In [23]:
# Target directories 
trainGenerator = train.flow_from_directory('../data/minc-2500/_train_', target_size = (224, 224), batch_size=batch_size,
    class_mode = "categorical", color_mode = "rgb", shuffle=True, seed=42)

testGenerator = test.flow_from_directory('../data/minc-2500/_test_', target_size = (224, 224), batch_size=1,
    class_mode = "categorical", color_mode = "rgb", shuffle=True, seed=42)

validationGenerator = test.flow_from_directory('../data/minc-2500/_valid_', target_size = (224, 224), batch_size=batch_size,
    class_mode = "categorical", color_mode = "rgb", shuffle=True, seed=42)

Found 7999 images belonging to 5 classes.
Found 2501 images belonging to 5 classes.
Found 2000 images belonging to 5 classes.


In [28]:
# Neural network
base = applications.mobilenet_v2.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

x = Flatten()(base.output)
x = Dropout(0.5)(x)
x = Dense(5, activation='softmax')(x)
model = Model(inputs=base.inputs, outputs=x)

In [18]:
# Print a summary of the network
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [30]:
# Set optimizer and loss function
opt = keras.optimizers.SGD(lr = 0.0001, momentum = 0.01)
optt = keras.optimizers.RMSprop(lr = 0.0001)
opt_adam = keras.optimizers.Adam(lr=0.0001, amsgrad=True)
model.compile(loss='categorical_crossentropy',
            optimizer=opt_adam,
            metrics=['categorical_accuracy'])

In [31]:
# callbacks
tensorboard = TensorBoard(log_dir="logs/minc-2500-{}-MobileNetV2-bs4".format(time()))
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00000001, min_delta=1e-2)
checkpoint = keras.callbacks.ModelCheckpoint('minc-2500-weights.{epoch:02d}-{val_loss:.2f}-MobileNetV2-bs4.hdf5', monitor='val_loss', 
        verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)

In [32]:
# Fit the model!
model.fit_generator(trainGenerator,
            steps_per_epoch = 2000,
            epochs = 50,
            validation_data = validationGenerator,
            validation_steps = 800,
            callbacks=[tensorboard, reduce_lr, checkpoint])

Epoch 1/50
2000/2000 [==============================] - 213s 107ms/step - loss: 1.6940 - categorical_accuracy: 0.4430 - val_loss: 1.0091 - val_categorical_accuracy: 0.6703

Epoch 00001: val_loss improved from inf to 1.00907, saving model to minc-2500-weights.01-1.01-MobileNetV2-bs4.hdf5
Epoch 2/50
2000/2000 [==============================] - 200s 100ms/step - loss: 1.2548 - categorical_accuracy: 0.5554 - val_loss: 0.8543 - val_categorical_accuracy: 0.6950

Epoch 00002: val_loss improved from 1.00907 to 0.85433, saving model to minc-2500-weights.02-0.85-MobileNetV2-bs4.hdf5
Epoch 3/50
2000/2000 [==============================] - 199s 100ms/step - loss: 1.1345 - categorical_accuracy: 0.5945 - val_loss: 1.1667 - val_categorical_accuracy: 0.6681

Epoch 00003: val_loss did not improve from 0.85433
Epoch 4/50
2000/2000 [==============================] - 201s 101ms/step - loss: 1.0368 - categorical_accuracy: 0.6242 - val_loss: 0.8161 - val_categorical_accuracy: 0.7156

Epoch 00004: val_loss i

2000/2000 [==============================] - 223s 112ms/step - loss: 0.6275 - categorical_accuracy: 0.7719 - val_loss: 0.6723 - val_categorical_accuracy: 0.7675

Epoch 00034: val_loss did not improve from 0.64849
Epoch 35/50
2000/2000 [==============================] - 218s 109ms/step - loss: 0.6394 - categorical_accuracy: 0.7606 - val_loss: 0.6952 - val_categorical_accuracy: 0.7678

Epoch 00035: val_loss did not improve from 0.64849
Epoch 36/50
2000/2000 [==============================] - 218s 109ms/step - loss: 0.6211 - categorical_accuracy: 0.7740 - val_loss: 0.6898 - val_categorical_accuracy: 0.7672

Epoch 00036: val_loss did not improve from 0.64849
Epoch 37/50
2000/2000 [==============================] - 219s 110ms/step - loss: 0.6357 - categorical_accuracy: 0.7631 - val_loss: 0.6582 - val_categorical_accuracy: 0.7788

Epoch 00037: val_loss did not improve from 0.64849
Epoch 38/50
2000/2000 [==============================] - 219s 109ms/step - loss: 0.6452 - categorical_accuracy: 

In [22]:
#from keras.models import load_model
#model = load_model('models/weights.12-0.17-DenseNet201.hdf5', compile=False)
#opt_adam = keras.optimizers.Adam(lr=0.0001, amsgrad=True)
model.compile(loss='categorical_crossentropy', optimizer=opt_adam, metrics=['categorical_accuracy', 
                                                                            'mean_squared_error', 
                                                                            'mean_absolute_error', 
                                                                            'mean_squared_logarithmic_error'])

result = model.evaluate_generator(testGenerator, steps=2000)
print(result)

[2.2257443563905372, 0.6085, 0.0641067344724917, 0.080268697268588, 0.03145471248751699]
